In [2]:
import boto3

s3_resource = boto3.resource('s3')

In [2]:
for bucket in s3_resource.buckets.all():
    print(bucket.name)

madhav-fantasy-football
madhav-glue-course


In [6]:
# Download a file from S3
s3_resource.Object(
    bucket_name='madhav-fantasy-football',
    key='database/top_performers/top_performers_by_year.csv'
).download_file('top_performers_by_year.csv')

In [7]:
# Upload a file to S3
s3_resource.Object(
    bucket_name='madhav-fantasy-football',
    key='database/top_performers/top_performers_by_year.csv.bk'
).upload_file('./top_performers_by_year.csv')

In [1]:
import boto3

from pyspark.sql import SparkSession
from delta import *

credentials = boto3.Session(profile_name='default').get_credentials()

# To find the correct hadoop version run
# spark._jvm.org.apache.hadoop.util.VersionInfo.getVersion()

# Upload a delta lake file to S3
builder = SparkSession.builder.appName('s3-upload') \
    .config('spark.jars.packages', 'org.apache.hadoop:hadoop-aws:3.3.4') \
    .config('spark.hadoop.fs.s3a.impl', 'org.apache.hadoop.fs.s3a.S3AFileSystem') \
    .config('spark.hadoop.fs.s3a.access.key', credentials.access_key) \
    .config('spark.hadoop.fs.s3a.secret.key', credentials.secret_key) \
    .config('spark.sql.extensions', 'io.delta.sql.DeltaSparkSessionExtension') \
    .config('spark.sql.catalog.spark_catalog', 'org.apache.spark.sql.delta.catalog.DeltaCatalog')

# Extra package will be downloaded to ~/.ivy2/jars
extra_packages = ['org.apache.hadoop:hadoop-aws:3.3.4']

spark = configure_spark_with_delta_pip(builder, extra_packages=extra_packages).getOrCreate()

23/12/02 19:42:21 WARN Utils: Your hostname, framework resolves to a loopback address: 127.0.1.1; using 10.0.0.72 instead (on interface wlp166s0)
23/12/02 19:42:21 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address


:: loading settings :: url = jar:file:/home/madhav/ws/pyspark-playground/.venv/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.1.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/madhav/.ivy2/cache
The jars for the packages stored in: /home/madhav/.ivy2/jars
io.delta#delta-spark_2.12 added as a dependency
org.apache.hadoop#hadoop-aws added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-266ab025-d42b-4def-8af1-64ceb86155b6;1.0
	confs: [default]
	found io.delta#delta-spark_2.12;3.0.0 in central
	found io.delta#delta-storage;3.0.0 in central
	found org.antlr#antlr4-runtime;4.9.3 in central
	found org.apache.hadoop#hadoop-aws;3.3.4 in central
	found com.amazonaws#aws-java-sdk-bundle;1.12.262 in central
	found org.wildfly.openssl#wildfly-openssl;1.0.7.Final in central
:: resolution report :: resolve 162ms :: artifacts dl 6ms
	:: modules in use:
	com.amazonaws#aws-java-sdk-bundle;1.12.262 from central in [default]
	io.delta#delta-spark_2.12;3.0.0 from central in [default]
	io.delta#delta-storage;3.0.0 from central in [default]
	org.antlr#antlr4-runtime;4.9.3 from central in [default]
	org.apache.hadoo

In [2]:
top_performers = spark.read.format('delta').load('s3a://madhav-fantasy-football/database/top_performers_delta')
top_performers.show()

23/12/02 19:43:54 WARN MetricsConfig: Cannot locate configuration: tried hadoop-metrics2-s3a-file-system.properties,hadoop-metrics2.properties
23/12/02 19:43:59 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


+----+--------+----------------+------------------+
|Year|Position|          Player|     FantasyPoints|
+----+--------+----------------+------------------+
|1999|      RB|    Emmitt Smith|             250.6|
|1999|      RB|   Curtis Martin|             247.3|
|1999|      RB|   Ricky Watters|             233.7|
|1999|      RB|     Duce Staley|             225.7|
|1999|      RB|    Corey Dillon|             212.0|
|1999|      RB|     Terry Kirby|            208.08|
|1999|      WR| Marvin Harrison|             351.7|
|1999|      WR|     Jimmy Smith|             313.6|
|1999|      WR|      Randy Moss|            294.68|
|1999|      WR|     Cris Carter|             292.1|
|1999|      WR| Marcus Robinson|             278.0|
|1999|      WR|     Isaac Bruce|             268.7|
|1999|      WR| Muhsin Muhammad|             267.3|
|1999|      WR|       Tim Brown|260.79999999999995|
|1999|      WR| Germane Crowell|             258.6|
|1999|      WR|Keyshawn Johnson|             254.6|
|1999|      

In [8]:
top_performers.printSchema()
top_performers.describe().show()

root
 |-- Year: long (nullable = true)
 |-- Position: string (nullable = true)
 |-- Player: string (nullable = true)
 |-- FantasyPoints: double (nullable = true)



+-------+------------------+--------+--------------+-----------------+
|summary|              Year|Position|        Player|    FantasyPoints|
+-------+------------------+--------+--------------+-----------------+
|  count|              1764|    1764|          1764|             1764|
|   mean|1995.4285714285713|    NULL|          NULL|245.3946825396825|
| stddev|14.855858776545205|    NULL|          NULL|58.80884799652613|
|    min|              1970|      QB|    A.J. Brown|            96.28|
|    max|              2020|      WR|Yancey Thigpen|            481.1|
+-------+------------------+--------+--------------+-----------------+



In [10]:
# Create an empty dataframe and save it to top_performers_delta
top_performers = spark.createDataFrame([], top_performers.schema)
top_performers.show()

+----+--------+------+-------------+
|Year|Position|Player|FantasyPoints|
+----+--------+------+-------------+
+----+--------+------+-------------+



In [ ]:
# Another way to return an empty dataframe with the same schema
# top_performers.limit(0)

In [11]:
# Write the dataframe back to S3
top_performers.write.format('delta').mode('overwrite').save('s3a://madhav-fantasy-football/database/top_performers_delta')

In [12]:
# Check that the dataframe is truly empty by reading it one more time
top_performers = spark.read.format('delta').load('s3a://madhav-fantasy-football/database/top_performers_delta')

In [13]:
top_performers.show()

+----+--------+------+-------------+
|Year|Position|Player|FantasyPoints|
+----+--------+------+-------------+
+----+--------+------+-------------+

